# Demo 02 - Adaptive Spectrum Processing

This notebook presents an example to read in a sweep of raw data and perform the Adaptive Spectrum Processing method, which was published by Kong++12. The original algorithm description is documented in the paper:

\[Kong++12\] Kong, F., Y. Zhang, R. Palmer, "Wind Turbine Clutter Mitigation for Weather Radar by Adaptive Spectrum Processing," _2012 IEEE Radar Conference_, Atlanta, GA, 2012.

In [ ]:
import os
import time
import glob
import numpy as np
import scipy as sp
import scipy.signal
import matplotlib
import matplotlib.patheffects
import matplotlib.pyplot as plt
import random

import toshi
import cspec

# plt.style.use('./darkmode.style')
plt.style.use('./lightmode.style')
zmap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', cspec.colormap.zmap()[:, :3])
vmap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', cspec.colormap.vmap()[:, :3], 64)
imap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', cspec.colormap.c12map()[:, :3], 12)
bmap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', cspec.colormap.i2map()[:, :3], 2)

path_effects = [
    matplotlib.patheffects.Stroke(linewidth=1.6, foreground=(0, 0, 0, 0.8)),
    matplotlib.patheffects.Normal()
]

## External Data

- Waveforms for pulse compression is stored under `blob/` and read in through `toshi.tx_waveform()`
- A table of wind turbine locations is stored under `blob/` and read in through `toshi.wtc_loc_from_csv()`

In [ ]:
waveform = toshi.tx_waveform()
pc_nfft = 1024
wf = sp.fft.fft(waveform / np.sqrt(np.sum(np.abs(waveform) ** 2)), n=pc_nfft)

wtc_loc = toshi.wtc_loc_from_csv()
turbines = np.array([[pt[2], pt[1]] for pt in wtc_loc], dtype=np.single)

## Data Tree

In [ ]:
data_home = os.path.expanduser('~/Downloads/toshiba/AKITA_IQ_TO_OU/IQdata')
files = glob.glob('{}/**/*.dat'.format(data_home), recursive=True)
files = sorted(files)

In [ ]:
file = files[0]
filesize = os.path.getsize(file)
basename = os.path.basename(file)
print('{} - {:,.0f} B'.format(file, filesize))

prefix = os.path.expanduser('~/Downloads/{}'.format(os.path.basename(file)[:-3]))

s = time.time()
all_ray_pulses, all_cpi_headers = toshi.read(file)
e = time.time()
print('{} read in {:.2f} s'.format(basename, e - s))

## Gather Basic Parameters

In [ ]:
# Go through the pulses for azimuth
az = np.zeros(len(all_ray_pulses), dtype=np.single)
for k, pulses in enumerate(all_ray_pulses):
    az[k] = pulses[0].azimuth
    
# Choose ray 2 to whatever that completes the 360-deg coverage
k = np.argmin(np.abs(az[3:] - az[2])) + 1
az = az[2:k+2]
ray_pulses = all_ray_pulses[2:k+2]
cpi_headers = all_cpi_headers[2:k+2]

# Dimensions
naz = len(ray_pulses)
ngate_long = ray_pulses[0][0].cpi_header.num_range_long_hi
ngate_short_hi = ray_pulses[0][0].cpi_header.num_range_short_hi
ngate_short_lo = ray_pulses[0][0].cpi_header.num_range_short_lo
ngate = ngate_long + ngate_short_hi + ngate_short_lo

# Elevation assumed to be flat from the very first pulse
scan_el = ray_pulses[0][0].elevation
scan_time = time.strptime(os.path.basename(file)[:15], '%Y%m%d_%H%M%S')

# Sampling code from the CPI header
fs = 1.0e6 * (1 << cpi_headers[0].fs_code)
dr = 3.0e8 / fs / 2
r = 1.0e-3 * (np.arange(0, ngate, dtype=np.single) * dr + 0.5 * dr)

print('fs = {:,.0f} Hz'.format(fs))
print('dr = {:.1f} m'.format(dr))
print('k = {} -> 2 ... {} ({})'.format(k, k+2, naz))

# Noise estimate, try azimuth 0, around 20-25 km
# ia = np.argmin(np.abs(a[3:] - 0.0)) + 1
# ir, er = np.argmin(np.abs(r - 20.0)), np.argmin(np.abs(r - 25.0))
# samples = np.zeros((len(ray_pulses[ia]), er-ir), dtype=np.csingle)

# Gather the samples. Ignore phase code since we are only interested in amplitude
# for k, pulse in enumerate(ray_pulses[ia]):
#     samples[k, :] = pulse.h_long_hi[ir:er]
# noise = np.mean(np.abs(samples)) ** 2

# Hard code noise estimate to be about 24 (eye ball)
noise = 24
print('Using noise estimate in 16-bit ADU: {:.4f}'.format(noise))

## Pulse Compression and Moment Processing

The process can be summarized as follows:

 - Contaminated coordinates are imported from an external file.
   - Each coordinate comes in (azimuth, range) that indicates the presence of wind turbines.
   - The external is tabulated in a CSV (comma separated value) entries
 - The clean area is derived from expanding the contaminated area using dilation.
   - Recursive dilation can be applied to expand the affected cells
   - This is necessary as the radar beam smears the contamination
   - Clean cells are selected just outside of the contaminated area
   - _NEW_: a pseudo-random shuffle is performed to uniformly spread out the selection
 - Contaminated areas are clustered using the simple pixel-to-pixel water-shed like algorithm.
 - Average filter is derived using in spectral domain using signal spectra, incoherently averaged.
   - _NEW_ for practicality: a further smoothing is accomplished using a rolling average, m-tap.
 

In [ ]:
s = np.zeros((naz, ngate), dtype=np.single)
v = np.zeros((naz, ngate), dtype=np.single)

nfft = 256
use_window = True
use_pulse_pair = False
use_dc_removal = False
omega = np.arange(-nfft/2, nfft/2) / nfft * np.pi                          # Frequency axis

# Go through the pulses
cpuls = []
for k, (pulses, cpi_header) in enumerate(zip(ray_pulses, cpi_headers)):
    npulse = len(pulses)

    # Decode the long pulse, then compress using wf
    p = np.zeros((npulse, ngate_long), dtype=np.csingle)
    for j, pulse in enumerate(pulses):
        p[j, :] = pulse.h_long_hi * np.exp(-1j * pulse.phase_h_long)       # Phase decoding
    pf = sp.fft.fft(p, n=pc_nfft, axis=1)                                  # Pulse compression in Fourier domain
    pc = sp.fft.ifft(pf * wf, n=pc_nfft, axis=1)                           # Return to time domain

    # Gather the short pulses and the compressed long pulses
    p = np.zeros((npulse, ngate), dtype=np.csingle)
    for j, pulse in enumerate(pulses):
        c = np.exp(-1j * pulse.phase_h_short)                              # Phase code
        p[j, :ngate_long] = pc[j, :ngate_long]                             # Long only
        p[j, :ngate_short_hi] = pulse.h_short_hi * c                       # Short hi
        p[j, :ngate_short_lo] = pulse.h_short_lo * c                       # Short lo        

    if use_dc_removal:
        p = p - np.mean(p, axis=0)                                         # Remove DC if desired

    cpuls.append(p)                                                        # Keep a copy of the compressed pulse

    if use_window:
        w = scipy.signal.get_window('blackmanharris', p.shape[0])
    else:
        w = np.ones((p.shape[0],))
    w /= np.sqrt(np.sum(w ** 2)) / np.sqrt(p.shape[0])                     # Normalize to non-windowed gain
    ww = np.repeat(np.expand_dims(w, axis=1), ngate, axis=1)               # Make same shape
    p *= ww                                                                # Windowing

    if use_pulse_pair:
        pp = p[1:, :] * np.conj(p[:-1, :])                                 # x(n) * x'(n-1)
        s[k, :] = np.mean(np.abs(p) ** 2, axis=0)                          # s(n) = E[x(n) * x'(n)]
        v[k, :] = np.angle(np.sum(pp, axis=0))                             # r(1) = E[x(n) * x'(n-1)]
    else:
        spec = np.fft.fft(p, nfft, axis=0)                                 # FFT -> Periodogram
        s[k, :] = np.mean(np.abs(spec) ** 2, axis=0) / p.shape[0]          # Signal power
        v[k, :] = np.angle(np.fft.ifft(spec * np.conj(spec), axis=0)[1])   # IFT -> ACF[1]

# Signal
s -= noise
s[s <= 0] = 1.0e-6                                                         # Avoid log(0)
snr = 10 * np.log10(s / noise)                                             # Signal-to-noise ratio in dB
z = 10 * np.log10(s * (r + 0.5e-3 * dr) ** 2) - 40                         # Estimated ZCal = -40 
z[:, :ngate_short_hi] += 15                                                # ~15-dB on the short waveform, perhaps?

# Proper velocity unfolding is not of concern here so just replace odd rays with even rays
v[1::2, :] = v[::2, :]

# Thresholding at SNR = 0 dB
o = snr < -1
z[o] = np.nan
v[o] = np.nan

## Chart

In [ ]:
def show_chart(img, cmap=imap, show_turbine=False,
               xlim=(-18, 18), ylim=(-18, 18), clim=(0.75, 6.75),
               grid_color=(0.5, 0.5, 0.5), isbool=False, ticklabels=None,
               title='Title'):
    plt.figure(figsize=(7.5, 6), dpi=144)
    plt.pcolormesh(xx, yy, img, cmap=cmap, shading='flat')
    plt.plot(ring_x, ring_y, color=grid_color, linewidth=0.75)
    plt.plot(cross_x, cross_y, color=grid_color, linewidth=0.75)
    if show_turbine:
        plt.plot(x_turb, y_turb, 'xk', markersize=4)
    for i in range(len(ring_r) - 1):
        plt.text(ring_x[9, i], ring_y[9, i], '{} km'.format(ring_r[i]), 
                 fontsize=9, fontweight='bold', color='w', path_effects=path_effects,
                 ha='center', va='center', rotation=-17)
    if isbool:
        plt.clim((-0.5, 1.5))
        c = plt.colorbar()
        c.set_ticks((0, 1))
        c.set_ticklabels(('', 'WTC'))       
    else:
        plt.clim(clim)
        plt.colorbar()
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.title(title)
    plt.show()

In [ ]:
# Edge of range cells
wid_a = np.mean(sorted(np.diff(az))[int(0.3 * len(az)):int(0.6 * len(az))])
end_a = az[-1] + wid_a
if end_a >= 360.0:
    end_a -= 360.0
ae = np.append(az, end_a)
re = 1.0e-3 * np.arange(0, ngate + 1, dtype=np.single) * dr

# Radar cell locations
ce = np.cos(np.deg2rad(scan_el))
rr, aa = np.meshgrid(re, np.deg2rad(ae))
xx = rr * ce * np.sin(aa)
yy = rr * ce * np.cos(aa)

# Rings and crosses
ring_r = np.arange(5, 21, 5)
ring_a = np.arange(0, 361, 2) / 180 * np.pi
ring_x = np.outer(np.sin(ring_a), ring_r)
ring_y = np.outer(np.cos(ring_a), ring_r)
cross_r = np.array([5, 50, np.nan])
cross_a = np.deg2rad(np.arange(0, 360, 45))
cross_x = np.outer(np.cos(cross_a), cross_r).flatten()
cross_y = np.outer(np.sin(cross_a), cross_r).flatten()

# Turbine locations
a_turb = turbines[:, 0] / 180.0 * np.pi
r_turb = turbines[:, 1]
x_turb = r_turb * np.sin(a_turb)
y_turb = r_turb * np.cos(a_turb)

# Show plots
show_chart(z, cmap=zmap, clim=(-32, 96), grid_color=(0, 0, 0, 0.6), title=basename, show_turbine=True)
show_chart(v, cmap=vmap, clim=(-5, 5), grid_color=(0, 0, 0, 0.6), title=basename)

## Wind Turbine Marking

In [ ]:
snr_thres = 3    # SNR threshold for clean cell selection
g = 20           # Maximum gates for clean cell selection
m = 3            # Tap count on spectral smoothing
n = 4            # Dilation recursion count

tags = cspec.wtc_tags(turbines, az, r, n=n)

In [ ]:
# Gather the 1st cluster
img = np.zeros(tags.shape)
for k in range(1, 5):
    emask = tags == k
    fmask = cspec.dilate(emask, n=n)
    gmask = cspec.dilate(fmask, n=2) ^ fmask

    # Cluster tags
    t = fmask + 1.5 * gmask
    o = t > 0
    img[o] = t[o] + (k - 1)

img[img < 1] = np.nan

show_chart(img, imap, title='Contaminated and Clean Cells')

## Single Cluster Processing

In [ ]:
itag = 3
emask = tags == itag
fmask = cspec.dilate(emask, n=n)
gmask = cspec.dilate(fmask, n=2) ^ fmask
gmask[snr < 3] = False

dirty_cells = cspec.mask2cells(fmask)
clean_cells = cspec.mask2cells(gmask)

# Gather all clean cells, shuffle the order
g = 20
ii = np.arange(len(clean_cells))
random.shuffle(ii)
selected_cells = []
for ic in ii[:g]:
    selected_cells.append(clean_cells[ic])
selected_mask = cspec.cell2mask(emask.shape, selected_cells)

# Show the clean cells
clean_mask = cspec.cell2mask(emask.shape, clean_cells)
clean_mask = np.array(clean_mask, dtype=np.single) * itag + 0.5
clean_mask[clean_mask < 1] = np.nan
clean_mask[selected_mask] = 6

# Average spectrm
spec = np.zeros(nfft, dtype=np.single)
for (ia, ir) in selected_cells:
    p = cpuls[ia][:, ir]
    w = scipy.signal.get_window('blackmanharris', len(p))
    #print((ia, ir), p.shape, w.shape)
    spec += np.abs(np.fft.fft(p * w, nfft))

# Add in a DC filter / GMAP-like zeroing
b = int(nfft * 0.025)
spec[:b] *= 1.0e-1
spec[-b:] *= 1.0e-1
spec /= np.sqrt(np.mean(spec ** 2))   # Normalize by noise gain

# m-tap circular averaging
filt = spec.copy()
for k in range(1, m):
    filt += np.roll(spec, k)
filt = np.roll(filt, -int(m/2))       # Compensate for running-average lag
filt /= m                             # Normalize by tap length

# Show plots
show_chart(clean_mask, title='Clean Cells and Selected Cells of Cluster #{}'.format(itag))

plt.figure(figsize=(8, 3), dpi=144)
plt.plot(omega, 20.0 * np.log10(np.fft.fftshift(filt)), label='1')
plt.xlim((-1.6, 1.6))
plt.ylim((-45, 16))
plt.grid(linewidth=0.25)
plt.xlabel('Omega (rad/sample)')
plt.ylabel('Filter Amplitude (dB)')
plt.title('Adaptive Filter Derived from the Clean Cells (m = {})'.format(m))
plt.show()

## More Gate Selections

In [ ]:
filts = []
gs = [10, 20, 40, 80]

for g in gs:
    spec = np.zeros(nfft, dtype=np.single)
    for (ia, ir) in clean_cells[:g]:
        p = cpuls[ia][:, ir]
        w = scipy.signal.get_window('blackmanharris', len(p))
        spec += np.abs(np.fft.fft(p * w, nfft))

    # Add in a DC filter / GMAP-like zeroing (could use something like resolution, len(p))
    spec /= np.sqrt(np.mean(spec ** 2))
    c = int(nfft * 0.02)
    spec[:c] = 1.0e-1
    spec[-c:] = 1.0e-1

    # m-tap circular averaging
    filt = spec.copy()
    for k in range(1, m):
        filt += np.roll(spec, k)
    filt = np.roll(filt, -int(m/2))       # Compensate for running-average lag
    filt /= m                             # Normalize by tap length

    filts.append(filt)

# Chart
plt.figure(figsize=(8, 3), dpi=144)
for k, g in enumerate(gs):
    plt.plot(omega, 20.0 * np.log10(np.fft.fftshift(filts[k])), label=g)
plt.xlim((-1.6, 1.6))
plt.ylim((-45, 16))
plt.grid(linewidth=0.25)
plt.xlabel('Omega (rad/sample)')
plt.ylabel('Filter Response (dB)')
plt.title('Adaptive Filter Derived from the Clean Cells')
plt.legend(gs)
plt.show()

## Adaptive Spectrum Processing for Multiple Clusters

In [ ]:
g = 30

sc = s.copy()
vc = v.copy()
use_window = True
last_tag = np.max(tags.flatten())

# Fixing random state for reproducibility
random.seed(20210301)

use = 0
for itag in range(1, last_tag+1):
    emask = tags == itag
    fmask = cspec.dilate(emask, n=n)
    gmask = cspec.dilate(fmask, n=2) ^ fmask
    gmask[snr < snr_thres] = False
    dirty_cells = cspec.mask2cells(fmask)
    clean_cells = cspec.mask2cells(gmask)

    # Gather all clean cells, shuffle the order
    ii = np.arange(len(clean_cells))
    random.shuffle(ii)
    x_clean_cells = []
    for ic in ii[:g]:
        x_clean_cells.append(clean_cells[ic])
    selected_mask = cspec.cell2mask(emask.shape, x_clean_cells)

    # Composite spectrum of the selected clean cells
    spec = np.zeros(nfft, dtype=np.single)
    for (ia, ir) in x_clean_cells:
        p = cpuls[ia][:, ir]
        w = scipy.signal.get_window('blackmanharris', len(p))
        spec += np.abs(np.fft.fft(p * w, nfft))
        #print((ia, ir), p.shape, w.shape, spec[0])

    # Add in a DC filter / GMAP-like zeroing
    spec /= np.sqrt(np.sum(spec ** 2))                                            # Normalize by noise gain
    c = int(nfft * 0.02)
    spec[:c+1] = 1.0e-1
    spec[-c:] = 1.0e-1
        
    # m-tap circular averaging
    filt = spec.copy()
    for k in range(1, m):
        filt += np.roll(spec, k)
    filt = np.roll(filt, -int(m/2))                                               # Compensate for running-average lag
    filt /= m                                                                     # Normalize by tap length
    print('cluster #{} filter gain = {:.2f} dB'.format(
        itag, 10.0 * np.log10(np.sum(filt ** 2))))

    # Go through the dirty cells
    for (ia, ir) in dirty_cells:
        p = cpuls[ia][:, ir]
        if use_window:
            w = scipy.signal.get_window('blackmanharris', len(p))
        else:
            w = np.ones((len(p),))
        w /= np.sqrt(np.sum(w ** 2)) / np.sqrt(len(p))                            # Normalize to non-windowed gain
        spec = np.fft.fft(p * w, nfft)                                            # FFT -> Periodogram
        spec *= filt                                                              # Filtering in Fourier domain
        tmp = np.mean(np.abs(spec) ** 2) / len(p)                                 # Signal power
        tmp -= noise
        # Raplace the cell if the filtered spectrum has lower overall power
        if s[ia, ir] > tmp:
            sc[ia, ir] = tmp
            vc[ia, ir] = np.angle(np.fft.ifft(spec * np.conj(spec), axis=0)[1])   # IFT -> ACF[1]
            use += 1
    
print('use = {}'.format(use))

# Signal
sc[sc <= 0] = 1.0e-6                                                              # Avoid log(0)
snr = 10 * np.log10(sc / noise)                                                   # Signal-to-noise ratio in dB
zc = 10 * np.log10(sc * (r + 0.5e-3 * dr) ** 2) - 40                              # Estimated ZCal = -40 
zc[:, :ngate_short_hi] += 15                                                      # ~15-dB on the short waveform, perhaps?

# Proper velocity unfolding is not of concern here so just replace odd rays with even rays
vc[1::2, :] = vc[::2, :]

# Thresholding
o = snr < -1
zc[o] = np.nan
vc[o] = np.nan

# Show plots
show_chart(zc, cmap=zmap, clim=(-32, 96), grid_color=(0, 0, 0, 0.6), title='Zc - {}'.format(basename))
show_chart(vc, cmap=vmap, clim=(-5, 5), grid_color=(0, 0, 0, 0.6), title='Vc - {}'.format(basename))

## Evaluation Cells

In [ ]:
emask = np.logical_and(tags > 0, np.isfinite(z))
# emask = cspec.dilate(emask, 4)
c2s = 10 * np.log10(sc[emask]) - 10 * np.log10(s[emask])
o = c2s < 0
c = np.sum(o)
delta = np.mean(c2s[o])

draw = 1.5 * emask + 1.5
draw[draw < 2] = np.nan
show_chart(draw, cmap=bmap, title='Turbine Cells ({:.2f} dB, {:,d} cells)'.format(delta, c), isbool=True)